In [2]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 823, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 823 (delta 36), reused 0 (delta 0), pack-reused 768
Receiving objects: 100% (823/823), 41.20 MiB | 32.25 MiB/s, done.
Resolving deltas: 100% (541/541), done.
Checking out files: 100% (411/411), done.


# Tarea 1: Riesgo de Mercado

In [3]:
%%R
#Instalamos los paquetes que necesitaremos.
install.packages("quantmod")
install.packages("data.table")
install.packages("PerformanceAnalytics")
install.packages("Deriv")

#Cargamos librerias
library(Deriv)
library(quantmod)
library(data.table)
library("PerformanceAnalytics")
library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [4]:
%%R
## GENERAMOS LAS FUNCIONES QUE NECESITAREMOS:

# Función de interpolación de tasas por el método alamabrada
talamb=function(nodos,curva,plazos){
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n){
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

# Función para diagonalizar un vector
diagv=function(x)	{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax){
    res[i,i]=x[i]
  }
  res
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 



In [5]:
%%R
#VaR con alisado

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 


## **Definición de parámetros para valorar**

In [6]:
%%R
fval=as.Date("20211015",format="%Y%m%d") #Fecha de valoración

In [7]:
%%R
alpha = 0.98 #@param {type:"slider", min:0, max:1, step:0.01}

In [8]:
%%R
#poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
itpl = 0 #@param ["0", "1"] {type:"raw"} 


### Inciso 1
1000 acciones de GCarso, -5000 de AméricaMóvil y 1200 WalMart. *Todas de la BMV*

In [9]:
%%R
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#Claves de cotización de las emisoras, en orden alfabético
pos_eq=c(-5000,1000,1200) #Monto inicial invertido en acciones
nh=3660 #Días de historia

### Inciso 2
1500 dólares (peso dólar), 700 euros, y -600 libras esterlinas.

In [10]:
%%R
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #Simbolos en orden alfabetico
pos_fx=c(700,-600,1500) #Monto incial invertido en divisas
nh=3660 #días de historia


### Inciso 3
2 bonos largos, uno cupón cero de cetes (con la curva guber) con nocional de 1500 vencimiento de 180 días, un bono M con tasa fija de 4.5% con un cupon de 4.05% anual vencimiento de 3600 días y un bono corto 1000 bondes con plazo 707 con cupón que paga 28 días (calcular cupón basado en curva de fondeo y descontar con la curva correspondiente). 

In [11]:
%%R

#CETES
base="RiesgosFinancieros/2022-1/Tarea/tasa_guber.txt"
ven_cete = cbind(180, 0)
contratos_cete = cbind(1500, 0)
nominal_cete = 10


#BONO M CON TASA FIJA
btasadesc="RiesgosFinancieros/2022-1/Tarea/tasa_yield.txt"
tf = cbind(0.045,0)
tfcupon = cbind(0.0405, 0)
ven_bonom = cbind(3600, 0)
plazocupon_bonom = cbind(182, 0)
contratos_bonom = cbind(1000, 0)
nominal_bonom = 100

#BONDES D
btasadescst="RiesgosFinancieros/2022-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2022-1/Tarea/tfondeo.txt"
plazos_bdm= cbind(707, 707) #Vencimiento del bono
plazocupon_bdm= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm = cbind(-1000, 0)
nominal_bdm=100

###Inciso 4
100 contratos de compra de futuros de peso dólar con un strike de 20.83 vencimiento de 5 días, y 50 contratos
de venta de futuros del IPC con strike de 49525 vencimiento de 53 días.

In [12]:
%%R
#FORWARDS TDC
bext="RiesgosFinancieros/2022-1/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2022-1/Tarea/tasa_fwd.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind(5,5)
contratos_fwd=cbind(100,0)
kst_fwd=cbind(20.83,20.83)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2022-1/Tarea/tasa_guber.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind(53,53)
contratos_fwd_ind=cbind(50,0)
kst_fwd_ind=cbind(49525,49525)
nominal_fwd_ind=1

### Inciso 5

Un swap largo nocional de 1600 pagando tasa fija de 4.6% anual (cada 28 días) y recibiendo tasa flotante de la curva TIIE vencimiento de 588 días, un swap corto con nocional de 1200 pagando tasa variable de la TIIE y recibiendo fija de 4.9% vencimiento de 270.

In [13]:
%%R
btasadesc_sw="RiesgosFinancieros/2022-1/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2022-1/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.046,0.049) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(1600,-1200) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1, 1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

### Inciso 6
Dos opciones europeas, una de tasa de interés larga call con strike de 5.8 % vencimiento de 1700 días 1000
contratos, y otra put larga de tasa de interés con strike de 6.0 % de la tiie nocional de 500 contratos vencimiento
700 días. (se valuará basado en las superficies de volatilidad si es in o out the money y en las curvas de tasa de interés TIIE o Pagarés).

In [14]:
%%R
btasadesc_oir="RiesgosFinancieros/2022-1/Tarea/tasa_TIIE_SW_OP.txt" #Tasa de descuento
btasaspot_oir="RiesgosFinancieros/2022-1/Tarea/tasa_DIRS_SW_OP.txt" #Tasa Spot
bvolspot_oir="RiesgosFinancieros/2022-1/Tarea/tvoltiie_opc.txt" #Volatilidad
plazos_oir=cbind( 1700, 700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor) 
K_oir=cbind( 0.058,0.06) #Strike
contratos_oir=cbind(1000,500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple

## **Carga de datos**

### Inciso 1

In [15]:
%%R
start_date=Sys.Date()-nh #fecha inicial
dataEnv<-new.env()
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)

bt.prep(dataEnv,align='remove.na',fill.gaps = T)
stock_prices = dataEnv$prices

###Inciso 2

In [16]:
%%R
  start_date=Sys.Date()-nh #fecha inicial
  dataEnvFX<-new.env() #Creamos donde guardar los datos
  getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date) #Obtenemos los datps
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T) #Limpieza de datos
  stock_pricesFX = dataEnvFX$prices #Nos da el precio

  stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))


### Inciso 3

In [17]:
%%R

# CETES
data_cetes<-read.table(base)
n_gov<-nrow(data_cetes)
m_gov=ncol(data_cetes)
x_orig_gov=data.frame(data_cetes[2:n_gov,1:m_gov])
x_orig_gov=as.data.table(mutate(data_cetes[2:n_gov,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data_cetes[1,2:m_gov])

# BONO M
data_m<-read.table(btasadesc)
n_bonom<-nrow(data_m)
m_bonom=ncol(data_m)
x_orig_bonom=data.frame(data_m[2:n_bonom,1:m_bonom])
x_orig_bonom=as.data.table(mutate(data_m[2:n_bonom,1:m_bonom],Date=as.Date(V1,format="%Y%m%d")))
x_orig_bonom=x_orig_bonom%>%select(-V1)
nodos_bonom=data.frame(data_m[1,2:m_bonom])

# BONDE D

data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
X_orig_bd=as.data.table(mutate(data1[2:n,1:m_bd],Date=as.Date(V1,format="%Y%m%d")))
X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])

X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 

#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.


          V1         V2         V3         V4
2 2021-10-15 0.04850092 0.04790516 0.04729400
3 2021-10-14 0.04737299 0.04790516 0.04729400
4 2021-10-13 0.04737299 0.04790516 0.04751145
5 2021-10-12 0.04793244 0.04790516 0.04751145
6 2021-10-11 0.04737299 0.04790516 0.04751145
7 2021-10-08 0.04737299 0.04786294 0.04751145


###Inciso 4


In [18]:
%%R
#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))

x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)


###Para Dolar

if (yext==1)
{
#Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial


  
#Creación del objeto para guardar los datos
  dataEnvFX_ftdc<-new.env()
  
#obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX_ftdc,from=start_date, to=(fval))
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX_ftdc,align='remove.na',fill.gaps=T)
  
#muestra de datos
  head(dataEnvFX_ftdc$prices[,2])
  
#Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_ftdc$prices[,2])),coredata(dataEnvFX_ftdc$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))

### Inciso 5

In [19]:
%%R
data1<-read.table(btasadesc_sw)
n1<-nrow(data1)
m1_orig_sw=ncol(data1)
X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X1_orig_sw=X1_orig_sw%>%select(-V1)
nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
data2<-read.table(btasacupvar_sw)
n2<-nrow(data2)
m2_orig_sw=ncol(data2)
X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X2_orig_sw=X2_orig_sw%>%select(-V1)
nodos2_sw=data.frame(data2[1,2:m2_orig_sw])


###Inciso 6

In [20]:
%%R
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])

##**Integración de Insumos**

### Cruce de tablas

In [21]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  

lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield

#lin_gub_aux=data.table(Date=as.Date(lin_gub[x_orig_bonom,on=.(Date),nomatch=0]$Date)) 

lin_gub_bmybdst_aux=data.table(Date=as.Date(lin_gub[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)


lin_gub_bmybdst=data.table(Date=as.Date(X3_ftdc[lin_gub_bmybdst_aux,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)

lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield,  st (bonde), libor

lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd

lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index



lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol


lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
print(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

           Date
  1: 2021-10-14
  2: 2021-10-13
  3: 2021-10-12
  4: 2021-10-11
  5: 2021-10-08
 ---           
249: 2020-10-20
250: 2020-10-19
251: 2020-10-16
252: 2020-10-15
253: 2020-10-14


### Formato de historia

#### Historia de todos

In [22]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
n

[1] 253


#### Incisos 1 y 2

In [23]:
%%R
#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#### Inciso 3

In [24]:
%%R
#Historia de curva gubernamental y yield
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#Historia de curvas de bonde
X1_orig=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)
X2_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X2_orig_bd=X2_orig_bd%>%select(-Date)
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)



#### Inciso 4


In [25]:
%%R
#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#### Inciso 5

In [26]:
%%R
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

####Inciso 6

In [27]:
%%R
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

## **Valuación**

### Inciso 1 y 2: Acciones y divisas

In [28]:
%%R
x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
m_fx=length(pos_fx)
m_acc=length(pos_eq)

### Inciso 3: Cetes

In [29]:
%%R
# Interpolamos las tasas
m=ncol(ven_cete)
x=matrix(0,n,m)
for (i in 1:(n))
{
    if(itpl == 0){
        x[i,]=approx(nodos_gov,x_orig_gov[i,], ven_cete)$y
    }else{
        x[i,]=talamb(nodos_gov,x_orig_gov[i,],ven_cete)
    }
}
#Función para el calculo de los bonos cupon 0
bonocupcero = function(i,t)
{
1/(1+i*t/360)
}

#Valor actual de cada bono
x0_cetes=x[1,]
V0_cetes=as.matrix(bonocupcero(x0_cetes,ven_cete))*nominal_cete 
V0_cetes[1,2] = 0
#Valor total
VT0_cetes=sum(V0_cetes)

print("Valor del cete")
VT0_cetes


[1] "Valor del cete"
[1] 9.760299


### Inciso 3: Bonos M

In [30]:
%%R
#interpolación de tasas y volatilidades
#revisar la valuacion que cambia si usamos n en vez de n-1
m=ncol(ven_bonom)
X=matrix(0,n,m)
for (i in 1:(n))
{
    if(itpl == 0){
        X[i,]=approx(nodos_bonom,x_orig_gov[i,], m)$y
    }else{
        X[i,]=talamb(nodos_bonom,x_orig_gov[i,],m)
    }
}

bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos = 1) #valoración bono tasa fija
{
x=x0  
N=as.integer(plazos/plazocupon)+1   
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0=0.045 #tasas de descuento valor actual
V0_bonom = bonoMyield(x0,ven_bonom, plazocupon_bonom, tfcupon, nominal_bonom, contratos_bonom)
V0_bonom[1,2] = 0
VT0_bonom=sum(V0_bonom)
print("Valor del Bono M")
VT0_bonom


[1] "Valor del Bono M"
[1] 96854.57


### Inciso 3: Bondes D

In [31]:
%%R
#BONDE D CÁLCULO
names(X2_orig_bd)[1] = 'Date'
X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 



V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
print("Valor del Bonde D")
print(V0_bd[1,1])


[1] "Valor del Bonde D"
[1] -97309.88


### Inciso 4: Forwards de tipo de cambio

In [32]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)


for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)
X_futtdc=cbind(X1_fwtdc,X2_fwtdc,X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd
# V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd]
V0_fwtdc

          [,1] [,2]
[1,] -26.92905    0


### Inciso 4: Forwards de indice

In [33]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m_ind=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m_ind) #DIVIDENDOS
X2_fwind=matrix(0,n,m_ind)

for (j in 1:n)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_find=as.matrix(X3_find)
X_futind=cbind(X1_fwind,X2_fwind,matrix(X3_find,n,ncol(X1_fwind)))

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
V0_fwind


         [,1] [,2]
[1,] 148027.6    0


### Inciso 5: Swaps

In [34]:
%%R
nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)
V0_sw

         [,1]      [,2]
[1,] 5.787402 0.7892475


###Inciso 6: Opciones

In [35]:
%%R
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}

x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#Valor del portafolio
V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir
V0_oir

         [,1]       [,2]
[1,] 1.422881 0.05877815


## Integración de factores de riesgo

In [36]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,8,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(X_bd_ext) #bonde
n_if[3]=ncol(X_sw) #swaps
n_if[4]=ncol(X_oir) #opciones tasa de interés
n_if[5]=ncol(X_futtdc) #Forwards de tipo de cambio
n_if[6]=ncol(X_futind) #Forwards de índices
n_if[7]=1 #Cetes
n_if[8]=0 #Bonos M
#valor del portafolios

V0_port=cbind(V0_acc_div, V0_bd, V0_sw, V0_oir,V0_fwtdc,V0_fwind, V0_cetes, VT0_bonom)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,X_bd_ext,X_sw,X_oir,X_futtdc,X_futind, x[,1], X[,1]) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos

DeltaX_s=DeltaX_port

## Medicion de riesgo, sin alisado

### Inciso 1 y 2: Acciones y divisas

In [37]:
%%R
m=m_fx+m_acc    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_fx)
Vfr2_acc_div=matrix(0,Ns,m_acc)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_fx)
PGfr2_acc_div=matrix(0,Ns,m_acc)
PGT_acc_div=matrix(0,Ns,1)
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

DeltaX_s_acc_div=DeltaX_s[,(1:n_if[1])]    #PASO CLAVE
x0_acc_div=stock_prices_EQFX[1,]   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*exp(DeltaX_s_acc_div[i,]))
  #PASO CLAVE
  V_acc_div[i,]=cbind(t(pos_fx),t(pos_eq))*X_s_acc_div[i,]
  #PASO CLAVE
  Vfr1_acc_div[i,]=t(pos_fx)*X_s_acc_div[i,1:m_fx]
  #PASO CLAVE
  Vfr2_acc_div[i,]=t(pos_eq)*X_s_acc_div[i,(m_fx+1):(m_fx+m_acc)]
  #PASO CLAVE
  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)
  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_fx])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_fx+1):(m_fx+m_acc)])
  PGT_acc_div[i,]=sum(PG_acc_div[i,])
  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}



PG_acc_div[1:5,]
PGfr1_acc_div[1:5,]
PGfr2_acc_div[1:5,]
PGT_acc_div[1:5,]


#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_fx)
VaRfr2_acc_div=matrix(0,1,m_acc)
CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_fx)
CVaRfr2_acc_div=matrix(0,1,m_acc)
for (i in (1:m))
{
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  if (i<=m_fx)
  {
  VaRfr1_acc_div[i]=equantile(PGfr1_acc_div[,i],1-alpha,Ns)
  CVaRfr1_acc_div[i]= mean(merge(which(PGfr1_acc_div[,i]<VaRfr1_acc_div[i]),cbind(seq(1,Ns),PGfr1_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
  if (i<=m_acc)
  {  
  VaRfr2_acc_div[i]=equantile(PGfr2_acc_div[,i],1-alpha,Ns)
  CVaRfr2_acc_div[i]= mean(merge(which(PGfr2_acc_div[,i]<VaRfr2_acc_div[i]),cbind(seq(1,Ns),PGfr2_acc_div[,i]), by.x=1,by.y=1)[,2])
  }
}

VaRCont_acc_div
VaRfr1_acc_div
VaRfr2_acc_div
CVaRCont_acc_div
CVaRfr1_acc_div
CVaRfr2_acc_div


#VaR Total

VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])


# print(cbind(VaRTotal_acc_div,sum(V0_acc_div), VaRCont_acc_div, V0_acc_div))
cbind(CVaRTotal_acc_div,sum(V0_acc_div), CVaRCont_acc_div, V0_acc_div)
cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div)
cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div)

print("Valor de acciones y divisas")
print(V0_acc_div )
print("Valor en Riesgo de acciones y divisas")
print(VaRCont_acc_div)
print("Valor en Riesgo de divisas")
print(VaRTotalfr1_acc_div)
print("Valor en Riesgo de acciones")
print(VaRTotalfr2_acc_div)

print("Valor en Riesgo Condicional de acciones y divisas")
print(CVaRCont_acc_div)
print("Valor en Riesgo Condicional de divisas")
print(CVaRTotalfr1_acc_div)
print("Valor en Riesgo Condicional de acciones")
print(CVaRTotalfr2_acc_div)


[1] "Valor de acciones y divisas"
   EURUSD.X  GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1:    16689 -16857.19 30830.55  -91250       73820     86592
[1] "Valor en Riesgo de acciones y divisas"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -229.3711 -254.1641 -457.1777 -2707.715 -3356.007 -3782.774
[1] "Valor en Riesgo de divisas"
[1] -559.876
[1] "Valor en Riesgo de acciones"
[1] -5066.12
[1] "Valor en Riesgo Condicional de acciones y divisas"
          [,1]      [,2]      [,3]      [,4]     [,5]    [,6]
[1,] -258.1004 -268.5236 -510.4965 -4132.774 -4133.76 -4322.2
[1] "Valor en Riesgo Condicional de divisas"
[1] -578.3648
[1] "Valor en Riesgo Condicional de acciones"
[1] -5853.425


### Inciso 3: Cetes

In [38]:
%%R
m = 1
X_s_cete=matrix(0,Ns,n_if[7])
PGfr1T_cete=matrix(0,Ns,1)
V_cetes=matrix(0,Ns,m)
Vfr1_cetes=matrix(0,Ns,m)
PG_cetes=matrix(0,Ns,m)


DeltaX_s_cete=DeltaX_s[,ncol(DeltaX_s)-1] 

for (i in 1:Ns)
{
  X_s_cete[i,]=x0_cetes[1]*exp(DeltaX_s_cete[i])
  V_cetes[i,]= bonocupcero(X_s_cete[i,1], ven_cete[1])
  PG_cetes[i,]=V_cetes[i,]-V0_cetes[1]
}

VaRCont_cetes=matrix(0,1,m)
CVaRCont_cetes=matrix(0,1,m)

for (i in (1:m))
{
  VaRCont_cetes[i]=equantile(PG_cetes[,i],1-alpha,Ns)
  CVaRCont_cetes[i]= mean(merge(which(PG_cetes[,i]<VaRCont_cetes[i]),cbind(seq(1,Ns),PG_cetes[,i]), by.x=1,by.y=1)[,2])
}

print("Valor del Cete")
print(V0_cetes[1,1])
print("Valor en riesgo del cete")
print(VaRCont_cetes)
print("Valor en Riesgo Condicional de cetes")
print(CVaRCont_cetes)


[1] "Valor del Cete"
[1] 9.760299
[1] "Valor en riesgo del cete"
          [,1]
[1,] -8.784688
[1] "Valor en Riesgo Condicional de cetes"
          [,1]
[1,] -8.784862


### Inciso 3: Bonos M
Dado que en el problema se nos está dando una tasa fija, la cual tenemos desde el momento en el que adquirimos el bono M. No se tiene riesgo, dado que desde el inicio ya sabemos como es que va evolucionando en cada periodo de tiempo.

### Inciso 3: Bondes D

In [39]:
%%R

#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(N_bd)    #PASO CLAVE
X_s_bd=matrix(0,Ns,n_if[2]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_bd=matrix(0,Ns,m)
Vfr1_bd=matrix(0,Ns,m)
Vfr2_bd=matrix(0,Ns,m)
Vfr3_bd=matrix(0,Ns,m)
PG_bd=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_bd=matrix(0,Ns,m)
PGfr2_bd=matrix(0,Ns,m)
PGfr3_bd=matrix(0,Ns,m)
PGT_bd=matrix(0,Ns,1)
PGfr1T_bd=matrix(0,Ns,1)
PGfr2T_bd=matrix(0,Ns,1)
PGfr3T_bd=matrix(0,Ns,1)

DeltaX_s_bd=DeltaX_s[,sum(n_if[1:1],1):sum(n_if[1:2])]    #PASO CLAVE
x0_bd=X_bd_ext[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_bd[i,]=x0_bd*exp(DeltaX_s_bd[i,])
  #PASO CLAVE
  V_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[2]/3+1):(n_if[2]*2/3)], X_s_bd[i,(n_if[2]*2/3+1):(n_if[2])], N_bd,ddv)
  #PASO CLAVE
  Vfr1_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, X_s_bd[i,1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[2]/3+1):(n_if[2]*2/3)], x0_bd[(n_if[2]*2/3+1):(n_if[2])], N_bd,ddv)
  #PASO CLAVE
  Vfr2_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, X_s_bd[i,(n_if[2]/3+1):(n_if[2]*2/3)], x0_bd[(n_if[2]*2/3+1):(n_if[2])], N_bd,ddv)
  #PASO CLAVE
  Vfr3_bd[i,]=bondeD(contratos_bdmT, nominal_bdm, x0_bd[1], plazocupon_bdmT, VTplazos_bdm, x0_bd[(n_if[2]/3+1):(n_if[2]*2/3)], X_s_bd[i,(n_if[2]*2/3+1):(n_if[2])], N_bd,ddv)
  #PASO CLAVE
  PG_bd[i,]=V_bd[i,]-V0_bd
  PGfr1_bd[i,]=Vfr1_bd[i,]-V0_bd
  PGfr2_bd[i,]=Vfr2_bd[i,]-V0_bd
  PGfr3_bd[i,]=Vfr3_bd[i,]-V0_bd
  PGT_bd[i,]=sum(PG_bd[i,])
  PGfr1T_bd[i,]=sum(PGfr1_bd[i,])
  PGfr2T_bd[i,]=sum(PGfr2_bd[i,])
  PGfr3T_bd[i,]=sum(PGfr3_bd[i,])
}


#VaR por posición
VaRCont_bd=matrix(0,1,m)
VaRfr1_bd=matrix(0,1,m)
VaRfr2_bd=matrix(0,1,m)
VaRfr3_bd=matrix(0,1,m)
CVaRCont_bd=matrix(0,1,m)
CVaRfr1_bd=matrix(0,1,m)
CVaRfr2_bd=matrix(0,1,m)
CVaRfr3_bd=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_bd[i]=equantile(PG_bd[,i],1-alpha,Ns)
  VaRfr1_bd[i]=equantile(PGfr1_bd[,i],1-alpha,Ns)
  VaRfr2_bd[i]=equantile(PGfr2_bd[,i],1-alpha,Ns)
  VaRfr3_bd[i]=equantile(PGfr3_bd[,i],1-alpha,Ns)
  CVaRfr1_bd[i]= mean(merge(which(PGfr1_bd[,i]<VaRfr1_bd[i]),cbind(seq(1,Ns),PGfr1_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_bd[i]= mean(merge(which(PGfr2_bd[,i]<VaRfr2_bd[i]),cbind(seq(1,Ns),PGfr2_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_bd[i]= mean(merge(which(PGfr3_bd[,i]<VaRfr3_bd[i]),cbind(seq(1,Ns),PGfr3_bd[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_bd[i]= mean(merge(which(PG_bd[,i]<VaRCont_bd[i]),cbind(seq(1,Ns),PG_bd[,i]), by.x=1,by.y=1)[,2])
}

#VaR Total

VaRTotal_bd=equantile(PGT_bd,1-alpha,Ns)
CVaRTotal_bd= mean(merge(which(PGT_bd<VaRTotal_bd),cbind(seq(1,Ns),PGT_bd), by.x=1,by.y=1)[,2])
VaRTotalfr1_bd=equantile(PGfr1T_bd,1-alpha,Ns)
CVaRTotalfr1_bd= mean(PGfr1T_bd[which(PGfr1T_bd<VaRTotalfr1_bd),])
VaRTotalfr2_bd=equantile(PGfr2T_bd,1-alpha,Ns)
CVaRTotalfr2_bd= mean(PGfr2T_bd[which(PGfr2T_bd<VaRTotalfr2_bd),])
VaRTotalfr3_bd=equantile(PGfr3T_bd,1-alpha,Ns)
CVaRTotalfr3_bd= mean(PGfr3T_bd[which(PGfr2T_bd<VaRTotalfr3_bd),])


print("Valor del bonde D")
print(V0_bd[1,1])
print("Valor en riesgo del bonde D")
print(VaRCont_bd[1,1])
print("Valor en Riesgo Condicional de bondes D")
print(CVaRCont_bd[1,1])
print(cbind(VaRTotalfr1_bd,VaRTotalfr2_bd,VaRTotalfr3_bd))
print(cbind(CVaRTotalfr1_bd,CVaRTotalfr2_bd,CVaRTotalfr3_bd))


[1] "Valor del bonde D"
[1] -97309.88
[1] "Valor en riesgo del bonde D"
[1] -363.9992
[1] "Valor en Riesgo Condicional de bondes D"
[1] -513.9109
     VaRTotalfr1_bd VaRTotalfr2_bd VaRTotalfr3_bd
[1,]      -251.0288      -160.0029      -69.44529
     CVaRTotalfr1_bd CVaRTotalfr2_bd CVaRTotalfr3_bd
[1,]       -466.2372       -233.6189       -57.61624


In [40]:
%%R
#Suma de perdidas y ganancias de los bonos largos y cortos.
PG_bonos = PG_cetes + PGT_bd
#VaR y CVaR total de los bonos
VaRT_bonos = equantile(PG_bonos,1-alpha,Ns)
CVaRT_bonos = mean(merge(which(PG_bonos<VaRT_bonos),cbind(seq(1,Ns),PG_bonos), by.x=1,by.y=1)[,2])

print(cbind(VaRT_bonos,CVaRT_bonos))

     VaRT_bonos CVaRT_bonos
[1,]  -372.7836   -522.6951


### Inciso 4: Forwards de tipo de cambio

In [41]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC
#dimensión
m=ncol(plazos_fwd)  #PASO CLAVE
X_s_fwtdc=matrix(0,Ns,n_if[5]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwtdc=matrix(0,Ns,m)
Vfr1_fwtdc=matrix(0,Ns,m)
Vfr2_fwtdc=matrix(0,Ns,m)
Vfr3_fwtdc=matrix(0,Ns,m)
PG_fwtdc=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwtdc=matrix(0,Ns,m)
PGfr2_fwtdc=matrix(0,Ns,m)
PGfr3_fwtdc=matrix(0,Ns,m)
PGT_fwtdc=matrix(0,Ns,1)
PGfr1T_fwtdc=matrix(0,Ns,1)
PGfr2T_fwtdc=matrix(0,Ns,1)
PGfr3T_fwtdc=matrix(0,Ns,1)

DeltaX_s_fwtdc=DeltaX_s[,sum(n_if[1:4],1):sum(n_if[1:5])]  #PASO CLAVE
x0_fwtdc=X_futtdc[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_fwtdc[i,]=x0_fwtdc*exp(DeltaX_s_fwtdc[i,])
  #PASO CLAVE
  V_fwtdc[i,]=futuroTC(plazos_fwd,X_s_fwtdc[i,1:((n_if[5]-1)/2)],X_s_fwtdc[i,((n_if[5]-1)/2+1):(n_if[5]-1)],X_s_fwtdc[i,(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd     
  #PASO CLAVE
  Vfr1_fwtdc[i,]=futuroTC(plazos_fwd,X_s_fwtdc[i,1:((n_if[5]-1)/2)],x0_fwtdc[((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd
  #PASO CLAVE
  Vfr2_fwtdc[i,]=futuroTC(plazos_fwd,x0_fwtdc[1:((n_if[5]-1)/2)],X_s_fwtdc[i,((n_if[5]-1)/2+1):(n_if[5]-1)],x0_fwtdc[(n_if[5])],kst_fwd)*contratos_fwd*nominal_fwd 
  PG_fwtdc[i,]=V_fwtdc[i,]-V0_fwtdc
  PGfr1_fwtdc[i,]=Vfr1_fwtdc[i,]-V0_fwtdc
  PGfr2_fwtdc[i,]=Vfr2_fwtdc[i,]-V0_fwtdc
  PGfr3_fwtdc[i,]=Vfr3_fwtdc[i,]-V0_fwtdc
  PGT_fwtdc[i,]=sum(PG_fwtdc[i,])
  PGfr1T_fwtdc[i,]=sum(PGfr1_fwtdc[i,])
  PGfr2T_fwtdc[i,]=sum(PGfr2_fwtdc[i,])
  PGfr3T_fwtdc[i,]=sum(PGfr3_fwtdc[i,])
}




#VaR por posición
VaRCont_fwtdc=matrix(0,1,m)
VaRfr1_fwtdc=matrix(0,1,m)
VaRfr2_fwtdc=matrix(0,1,m)
VaRfr3_fwtdc=matrix(0,1,m)
CVaRCont_fwtdc=matrix(0,1,m)
CVaRfr1_fwtdc=matrix(0,1,m)
CVaRfr2_fwtdc=matrix(0,1,m)
CVaRfr3_fwtdc=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_fwtdc[i]=equantile(PG_fwtdc[,i],1-alpha,Ns)
  VaRfr1_fwtdc[i]=equantile(PGfr1_fwtdc[,i],1-alpha,Ns)
  VaRfr2_fwtdc[i]=equantile(PGfr2_fwtdc[,i],1-alpha,Ns)
  VaRfr3_fwtdc[i]=equantile(PGfr3_fwtdc[,i],1-alpha,Ns)
  CVaRfr1_fwtdc[i]= mean(merge(which(PGfr1_fwtdc[,i]<VaRfr1_fwtdc[i]),cbind(seq(1,Ns),PGfr1_fwtdc[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_fwtdc[i]= mean(merge(which(PGfr2_fwtdc[,i]<VaRfr2_fwtdc[i]),cbind(seq(1,Ns),PGfr2_fwtdc[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_fwtdc[i]= mean(merge(which(PGfr3_fwtdc[,i]<VaRfr3_fwtdc[i]),cbind(seq(1,Ns),PGfr3_fwtdc[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_fwtdc[i]= mean(merge(which(PG_fwtdc[,i]<VaRCont_fwtdc[i]),cbind(seq(1,Ns),PG_fwtdc[,i]), by.x=1,by.y=1)[,2])
}



#VaR Total
VaRTotal_fwtdc=equantile(PGT_fwtdc,1-alpha,Ns)
CVaRTotal_fwtdc= mean(merge(which(PGT_fwtdc<VaRTotal_fwtdc),cbind(seq(1,Ns),PGT_fwtdc), by.x=1,by.y=1)[,2])
VaRTotalfr1_fwtdc=equantile(PGfr1T_fwtdc,1-alpha,Ns)
CVaRTotalfr1_fwtdc= mean(PGfr1T_fwtdc[which(PGfr1T_fwtdc<VaRTotalfr1_fwtdc),])
VaRTotalfr2_fwtdc=equantile(PGfr2T_fwtdc,1-alpha,Ns)
CVaRTotalfr2_fwtdc= mean(PGfr2T_fwtdc[which(PGfr2T_fwtdc<VaRTotalfr2_fwtdc),])
VaRTotalfr3_fwtdc=equantile(PGfr3T_fwtdc,1-alpha,Ns)
CVaRTotalfr3_fwtdc= mean(PGfr3T_fwtdc[which(PGfr2T_fwtdc<VaRTotalfr2_fwtdc),])


#cbind(VaRTotal_fwtdc,sum(V0_fwtdc), VaRCont_fwtdc, V0_fwtdc)
#print(V0_fwtdc)
#print(VaRCont_fwtdc)
#cbind(CVaRTotal_fwtdc,sum(V0_fwtdc), CVaRCont_fwtdc, V0_fwtdc)
#print(CVaRCont_fwtdc)
#print(CVaRfr1_fwtdc)
#print(CVaRfr2_fwtdc)
#print(CVaRfr3_fwtdc)
#print(cbind(VaRTotal_fwtdc,VaRTotalfr1_fwtdc,VaRTotalfr2_fwtdc,VaRTotalfr3_fwtdc))
#print(cbind(CVaRTotal_fwtdc,CVaRTotalfr1_fwtdc,CVaRTotalfr2_fwtdc,CVaRTotalfr3_fwtdc))

print("VaR por contrato")
print(VaRCont_fwtdc)
#print("VaR por factor de riesgo 1 por contrato")
#print(VaRfr1_fwtdc)
#print("VaR por factor de riesgo 2 por contrato")
#print(VaRfr2_fwtdc)
#print("VaR por factor de riesgo 3 por contrato")
#print(VaRfr3_fwtdc)#-
print("VaR por factor de riesgo 1 total")
print(VaRTotalfr1_fwtdc)
print("VaR por factor de riesgo 2 total")
print(VaRTotalfr2_fwtdc)
print("VaR por factor de riesgo 3 total")
print(VaRTotalfr3_fwtdc)
print("VaR Total")
print(VaRTotal_fwtdc)

print("CVaR por contrato")
print(CVaRCont_fwtdc)
#print("CVaR por factor de riesgo 1 por contrato")
#print(CVaRfr1_fwtdc)
#print("CVaR por factor de riesgo 2 por contrato")
#print(CVaRfr2_fwtdc)
#print("CVaR por factor de riesgo 3 por contrato")
#print(CVaRfr3_fwtdc)
print("CVaR por factor de riesgo 1 total")
print(CVaRTotalfr1_fwtdc)
print("CVaR por factor de riesgo 2 total")
print(CVaRTotalfr2_fwtdc)
print("CVaR por factor de riesgo 3 total")
print(CVaRTotalfr3_fwtdc)
print("CVaR Total")
print(CVaRTotal_fwtdc)





[1] "VaR por contrato"
         [,1] [,2]
[1,] -30.4725    0
[1] "VaR por factor de riesgo 1 total"
[1] -0.008600858
[1] "VaR por factor de riesgo 2 total"
[1] -0.01487834
[1] "VaR por factor de riesgo 3 total"
[1] 26.92905
[1] "VaR Total"
[1] -30.4725
[1] "CVaR por contrato"
          [,1] [,2]
[1,] -34.02239  NaN
[1] "CVaR por factor de riesgo 1 total"
[1] -0.01054397
[1] "CVaR por factor de riesgo 2 total"
[1] -0.01788901
[1] "CVaR por factor de riesgo 3 total"
[1] 26.92905
[1] "CVaR Total"
[1] -34.02239


### Inciso 4: Forwards de indice

In [42]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS IPC
m=ncol(plazos_fwd_ind)  #PASO CLAVE
X_s_fwind=matrix(0,Ns,n_if[6]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_fwind=matrix(0,Ns,m)
Vfr1_fwind=matrix(0,Ns,m)
Vfr2_fwind=matrix(0,Ns,m)
Vfr3_fwind=matrix(0,Ns,m)
PG_fwind=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_fwind=matrix(0,Ns,m)
PGfr2_fwind=matrix(0,Ns,m)
PGfr3_fwind=matrix(0,Ns,m)
PGT_fwind=matrix(0,Ns,1)
PGfr1T_fwind=matrix(0,Ns,1)
PGfr2T_fwind=matrix(0,Ns,1)
PGfr3T_fwind=matrix(0,Ns,1)

DeltaX_s_fwind=DeltaX_s[,sum(n_if[1:5],1):sum(n_if[1:6])]  #PASO CLAVE
x0_fwind=X_futind[1,] #PASO CLAVE


for (i in 1:Ns)
{
  X_s_fwind[i,]=x0_fwind* exp(DeltaX_s_fwind[i,])
  #PASO CLAVE
  V_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind     
  Vfr1_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  #PASO CLAVE
  Vfr2_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  #PASO CLAVE
  Vfr3_fwind[i,]=futuroTC(plazos_fwd_ind,X_s_fwind[i,1:(n_if[6]/3)],X_s_fwind[i,(n_if[6]/3+1):(n_if[6]*2/3)],X_s_fwind[i,(n_if[6]*2/3+1):n_if[6]],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind 
  PG_fwind[i,]=V_fwind[i,]-V0_fwind
  PGfr1_fwind[i,]=Vfr1_fwind[i,]-V0_fwind
  PGfr2_fwind[i,]=Vfr2_fwind[i,]-V0_fwind
  PGfr3_fwind[i,]=Vfr3_fwind[i,]-V0_fwind
  PGT_fwind[i,]=sum(PG_fwind[i,])
  PGfr1T_fwind[i,]=sum(PGfr1_fwind[i,])
  PGfr2T_fwind[i,]=sum(PGfr2_fwind[i,])
  PGfr3T_fwind[i,]=sum(PGfr3_fwind[i,])
}



#VaR por posición
VaRCont_fwind=matrix(0,1,m)
VaRfr1_fwind=matrix(0,1,m)
VaRfr2_fwind=matrix(0,1,m)
VaRfr3_fwind=matrix(0,1,m)
CVaRCont_fwind=matrix(0,1,m)
CVaRfr1_fwind=matrix(0,1,m)
CVaRfr2_fwind=matrix(0,1,m)
CVaRfr3_fwind=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_fwind[i]=equantile(PG_fwind[,i],1-alpha,Ns)
  VaRfr1_fwind[i]=equantile(PGfr1_fwind[,i],1-alpha,Ns)
  VaRfr2_fwind[i]=equantile(PGfr2_fwind[,i],1-alpha,Ns)
  VaRfr3_fwind[i]=equantile(PGfr3_fwind[,i],1-alpha,Ns)
  CVaRfr1_fwind[i]= mean(merge(which(PGfr1_fwind[,i]<VaRfr1_fwind[i]),cbind(seq(1,Ns),PGfr1_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_fwind[i]= mean(merge(which(PGfr2_fwind[,i]<VaRfr2_fwind[i]),cbind(seq(1,Ns),PGfr2_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_fwind[i]= mean(merge(which(PGfr3_fwind[,i]<VaRfr3_fwind[i]),cbind(seq(1,Ns),PGfr3_fwind[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_fwind[i]= mean(merge(which(PG_fwind[,i]<VaRCont_fwind[i]),cbind(seq(1,Ns),PG_fwind[,i]), by.x=1,by.y=1)[,2])
}
VaRCont_fwind
VaRfr1_fwind
VaRfr2_fwind
CVaRCont_fwind
CVaRfr1_fwind
CVaRfr2_fwind


#VaR Total

VaRTotal_fwind=equantile(PGT_fwind,1-alpha,Ns)
CVaRTotal_fwind= mean(merge(which(PGT_fwind<VaRTotal_fwind),cbind(seq(1,Ns),PGT_fwind), by.x=1,by.y=1)[,2])
VaRTotalfr1_fwind=equantile(PGfr1T_fwind,1-alpha,Ns)
CVaRTotalfr1_fwind= mean(PGfr1T_fwind[which(PGfr1T_fwind<VaRTotalfr1_fwind),])
VaRTotalfr2_fwind=equantile(PGfr2T_fwind,1-alpha,Ns)
CVaRTotalfr2_fwind= mean(PGfr2T_fwind[which(PGfr2T_fwind<VaRTotalfr2_fwind),])
VaRTotalfr3_fwind=equantile(PGfr3T_fwind,1-alpha,Ns)
CVaRTotalfr3_fwind= mean(PGfr3T_fwind[which(PGfr3T_fwind<VaRTotalfr3_fwind),])


print(cbind(VaRTotal_fwind,sum(V0_fwind), VaRCont_fwind, V0_fwind))
print(cbind(CVaRTotal_fwind,sum(V0_fwind), CVaRCont_fwind, V0_fwind))
print(cbind(VaRTotal_fwind,VaRTotalfr1_fwind,VaRTotalfr2_fwind,VaRTotalfr3_fwind))
cbind(CVaRTotal_fwind,CVaRTotalfr1_fwind,CVaRTotalfr2_fwind,CVaRTotalfr3_fwind)

print("VaR por contrato")
print(VaRCont_fwind)
#print("VaR por factor de riesgo 1 por contrato")
#print(VaRfr1_fwind)
#print("VaR por factor de riesgo 2 por contrato")
#print(VaRfr2_fwind)
#print("VaR por factor de riesgo 3 por contrato")
#print(VaRfr3_fwind)
print("VaR por factor de riesgo 1 total")
print(VaRTotalfr1_fwind)
print("VaR por factor de riesgo 2 total")
print(VaRTotalfr2_fwind)
print("VaR por factor de riesgo 3 total")
print(VaRTotalfr3_fwind)
print("VaR Total")
print(VaRTotal_fwind)

print("CVaR por contrato")
print(CVaRCont_fwind)
#print("CVaR por factor de riesgo 1 por contrato")
#print(CVaRfr1_fwind)
#print("CVaR por factor de riesgo 2 por contrato")
#print(CVaRfr2_fwind)
#print("CVaR por factor de riesgo 3 por contrato")
#print(CVaRfr3_fwind)
print("CVaR por factor de riesgo 1 total")
print(CVaRTotalfr1_fwind)
print("CVaR por factor de riesgo 2 total")
print(CVaRTotalfr2_fwind)
print("CVaR por factor de riesgo 3 total")
print(CVaRTotalfr3_fwind)
print("CVaR Total")
print(CVaRTotal_fwind)

     VaRTotal_fwind                                
[1,]      -47613.14 148027.6 -47613.14 0 148027.6 0
     CVaRTotal_fwind                                  
[1,]       -57376.18 148027.6 -57376.18 NaN 148027.6 0
     VaRTotal_fwind VaRTotalfr1_fwind VaRTotalfr2_fwind VaRTotalfr3_fwind
[1,]      -47613.14         -47613.14         -47613.14         -47613.14
[1] "VaR por contrato"
          [,1] [,2]
[1,] -47613.14    0
[1] "VaR por factor de riesgo 1 total"
[1] -47613.14
[1] "VaR por factor de riesgo 2 total"
[1] -47613.14
[1] "VaR por factor de riesgo 3 total"
[1] -47613.14
[1] "VaR Total"
[1] -47613.14
[1] "CVaR por contrato"
          [,1] [,2]
[1,] -57376.18  NaN
[1] "CVaR por factor de riesgo 1 total"
[1] -57376.18
[1] "CVaR por factor de riesgo 2 total"
[1] -57376.18
[1] "CVaR por factor de riesgo 3 total"
[1] -57376.18
[1] "CVaR Total"
[1] -57376.18


In [43]:
%%R
PGT_forwards = PGT_fwtdc+PGT_fwind

VaRT_forwards = equantile(PGT_forwards,1-alpha,Ns)
CVaRT_forwards = mean(merge(which(PGT_forwards<VaRT_forwards),cbind(seq(1,Ns),PGT_forwards), by.x=1,by.y=1)[,2])
print("VaR de futuros")
print(VaRT_forwards)
print("CVaR de futuros")
print(CVaRT_forwards)

[1] "VaR de futuros"
[1] -47607.9
[1] "CVaR de futuros"
[1] -57385.71


### Inciso 5: Swaps

In [44]:
%%R
#riesgo del swap
m=ncol(N)    #PASO CLAVE
X_s_sw=matrix(0,Ns,n_if[3]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_sw=matrix(0,Ns,m)
Vfr1_sw=matrix(0,Ns,m)
Vfr2_sw=matrix(0,Ns,m)
PG_sw=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_sw=matrix(0,Ns,m)
PGfr2_sw=matrix(0,Ns,m)
PGT_sw=matrix(0,Ns,1)
PGfr1T_sw=matrix(0,Ns,1)
PGfr2T_sw=matrix(0,Ns,1)

DeltaX_s_sw=DeltaX_s[,sum(n_if[1:2],1):sum(n_if[1:3])]    #PASO CLAVE
x0_sw=as.numeric(c(XtfwdT[1,],Xvp[1,]))   #PASO CLAVE

for (i in 1:Ns)
{
  X_s_sw[i,]=x0_sw*exp(DeltaX_s_sw[i,])
  #PASO CLAVE
  V_sw[i,]=swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  Vfr1_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,X_s_sw[i,1:(n_if[3]/2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,],N)
  #PASO CLAVE
  Vfr2_sw[i,]=swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[3]/2+1):(n_if[3])],N)
  #PASO CLAVE
  PG_sw[i,]=V_sw[i,]-V0_sw
  PGfr1_sw[i,]=Vfr1_sw[i,]-V0_sw
  PGfr2_sw[i,]=Vfr2_sw[i,]-V0_sw
  PGT_sw[i,]=sum(PG_sw[i,])
  PGfr1T_sw[i,]=sum(PGfr1_sw[i,])
  PGfr2T_sw[i,]=sum(PGfr2_sw[i,])
}



#VaR por posición
VaRCont_sw=matrix(0,1,m)
VaRfr1_sw=matrix(0,1,m)
VaRfr2_sw=matrix(0,1,m)
CVaRCont_sw=matrix(0,1,m)
CVaRfr1_sw=matrix(0,1,m)
CVaRfr2_sw=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_sw[i]=equantile(PG_sw[,i],1-alpha,Ns)
  VaRfr1_sw[i]=equantile(PGfr1_sw[,i],1-alpha,Ns)
  VaRfr2_sw[i]=equantile(PGfr2_sw[,i],1-alpha,Ns)
  CVaRfr1_sw[i]= mean(merge(which(PGfr1_sw[,i]<VaRfr1_sw[i]),cbind(seq(1,Ns),PGfr1_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_sw[i]= mean(merge(which(PGfr2_sw[,i]<VaRfr2_sw[i]),cbind(seq(1,Ns),PGfr2_sw[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_sw[i]= mean(merge(which(PG_sw[,i]<VaRCont_sw[i]),cbind(seq(1,Ns),PG_sw[,i]), by.x=1,by.y=1)[,2])
}

#VaR Total

VaRTotal_sw=equantile(PGT_sw,1-alpha,Ns)
CVaRTotal_sw= mean(merge(which(PGT_sw<VaRTotal_sw),cbind(seq(1,Ns),PGT_sw), by.x=1,by.y=1)[,2])
VaRTotalfr1_sw=equantile(PGfr1T_sw,1-alpha,Ns)
CVaRTotalfr1_sw= mean(PGfr1T_sw[which(PGfr1T_sw<VaRTotalfr1_sw),])
VaRTotalfr2_sw=equantile(PGfr2T_sw,1-alpha,Ns)
CVaRTotalfr2_sw= mean(PGfr2T_sw[which(PGfr2T_sw<VaRTotalfr2_sw),])


print("VaR por contrato")
print(VaRCont_sw)
print("VaR por factor de riesgo 1 por contrato")
print(VaRfr1_sw)
print("VaR por factor de riesgo 2 por contrato")
print(VaRfr2_sw)
print("VaR por factor de riesgo 1 total")
print(VaRTotalfr1_sw)
print("VaR por factor de riesgo 2 total")
print(VaRTotalfr2_sw)
print("VaR Total")
print(VaRTotal_sw)

print("CVaR por contrato")
print(CVaRCont_sw)
print("CVaR por factor de riesgo 1 por contrato")
print(CVaRfr1_sw)
print("CVaR por factor de riesgo 2 por contrato")
print(CVaRfr2_sw)
print("CVaR por factor de riesgo 1 total")
print(CVaRTotalfr1_sw)
print("CVaR por factor de riesgo 2 total")
print(CVaRTotalfr2_sw)
print("CVaR Total")
print(CVaRTotal_sw)


[1] "VaR por contrato"
         [,1]       [,2]
[1,] -2.23043 -0.5421164
[1] "VaR por factor de riesgo 1 por contrato"
          [,1]       [,2]
[1,] -2.230764 -0.5421191
[1] "VaR por factor de riesgo 2 por contrato"
             [,1]          [,2]
[1,] -0.001108068 -0.0001020942
[1] "VaR por factor de riesgo 1 total"
[1] -2.772883
[1] "VaR por factor de riesgo 2 total"
[1] -0.001210162
[1] "VaR Total"
[1] -2.772532
[1] "CVaR por contrato"
          [,1]       [,2]
[1,] -3.036852 -0.8229418
[1] "CVaR por factor de riesgo 1 por contrato"
         [,1]       [,2]
[1,] -3.43931 -0.8229594
[1] "CVaR por factor de riesgo 2 por contrato"
            [,1]          [,2]
[1,] -0.00279425 -0.0002501779
[1] "CVaR por factor de riesgo 1 total"
[1] -4.197579
[1] "CVaR por factor de riesgo 2 total"
[1] -0.003044428
[1] "CVaR Total"
[1] -3.72305


### Inciso 6: Opciones

In [45]:
%%R
m=ncol(plazos_oir)  #PASO CLAVE
X_s_oir=matrix(0,Ns,n_if[4]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s[,sum(n_if[1:3],1):sum(n_if[1:4])]  #PASO CLAVE
x0_oir=X_oir[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*exp(DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],X_s_oir[i,((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,x0_oir[((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[4]/3))],x0_oir[((n_if[4]/3+1):(n_if[4]/3*2))],K_oir,X_s_oir[i,((n_if[4]*2/3+1):(n_if[4]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_oir
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}



#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_oir[i]=equantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=equantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=equantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=equantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}


#VaR Total
VaRTotal_oir=equantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=equantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=equantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=equantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])

print(V0_oir)
print("VaR por contrato")
print(VaRCont_oir)
print("VaR por factor de riesgo 1 por contrato")
print(VaRfr1_oir)
print("VaR por factor de riesgo 2 por contrato")
print(VaRfr2_oir)
print("VaR por factor de riesgo 3 por contrato")
print(VaRfr3_oir)
print("VaR por factor de riesgo 1 total")
print(VaRTotalfr1_oir)
print("VaR por factor de riesgo 2 total")
print(VaRTotalfr2_oir)
print("VaR por factor de riesgo 3 total")
print(VaRTotalfr3_oir)
print("VaR Total")
print(VaRTotal_oir)

print("CVaR por contrato")
print(CVaRCont_oir)
print("CVaR por factor de riesgo 1 por contrato")
print(CVaRfr1_oir)
print("CVaR por factor de riesgo 2 por contrato")
print(CVaRfr2_oir)
print("CVaR por factor de riesgo 3 por contrato")
print(CVaRfr3_oir)
print("CVaR por factor de riesgo 1 total")
print(CVaRTotalfr1_oir)
print("CVaR por factor de riesgo 2 total")
print(CVaRTotalfr2_oir)
print("CVaR por factor de riesgo 3 total")
print(CVaRTotalfr3_oir)
print("CVaR Total")
print(CVaRTotal_oir)


         [,1]       [,2]
[1,] 1.422881 0.05877815
[1] "VaR por contrato"
            [,1]         [,2]
[1,] -0.07138323 -0.009956681
[1] "VaR por factor de riesgo 1 por contrato"
             [,1]         [,2]
[1,] -0.007014216 -8.08202e-05
[1] "VaR por factor de riesgo 2 por contrato"
            [,1]         [,2]
[1,] -0.06964301 -0.009956374
[1] "VaR por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]    0    0
[1] "VaR por factor de riesgo 1 total"
[1] -0.007074378
[1] "VaR por factor de riesgo 2 total"
[1] -0.06515239
[1] "VaR por factor de riesgo 3 total"
[1] 0
[1] "VaR Total"
[1] -0.06515247
[1] "CVaR por contrato"
            [,1]        [,2]
[1,] -0.09031513 -0.01435228
[1] "CVaR por factor de riesgo 1 por contrato"
             [,1]          [,2]
[1,] -0.007421767 -0.0001539431
[1] "CVaR por factor de riesgo 2 por contrato"
            [,1]        [,2]
[1,] -0.09030153 -0.01435201
[1] "CVaR por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]  NaN  NaN
[1] "CVaR por 

## Medición de riesgo, con alisado

### Generación de escenarios

In [46]:
%%R
w0 <- 0.05 #@param {type:"slider", min:0, max:1, step:0.01}
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root


#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}

p_esc=genera_esc(lambda,w0,Ns)

#### Inciso 1 y 2: Acciones y divisas


In [47]:
%%R
m=m_fx+m_acc 
VaRCACont_acc_div=matrix(0,1,m)
VaRCAfr1_acc_div=matrix(0,1,m_fx)
VaRCAfr2_acc_div=matrix(0,1,m_acc)
CVaRCACont_acc_div=matrix(0,1,m)
CVaRCAfr1_acc_div=matrix(0,1,m_fx)
CVaRCAfr2_acc_div=matrix(0,1,m_acc)
for (i in (1:m))
{
  VaRCACont_acc_div[i]=wquantile(as.matrix(PG_acc_div[,i]), p_esc, 1-alpha)
  CVaRCACont_acc_div[i]= wcvar(as.matrix(PG_acc_div[,i]), p_esc, 1-alpha)
  if (i<=m_fx)
  {
  VaRCAfr1_acc_div[i]=wquantile(as.matrix(PGfr1_acc_div[,i]), p_esc, 1-alpha)
  CVaRCAfr1_acc_div[i]= wcvar(as.matrix(PGfr1_acc_div[,i]), p_esc, 1-alpha)
  }
  if (i<=m_acc)
  {  
  VaRCAfr2_acc_div[i]=wquantile(as.matrix(PGfr2_acc_div[,i]), p_esc, 1-alpha)
  CVaRCAfr2_acc_div[i]= wcvar(as.matrix(PGfr2_acc_div[,i]), p_esc, 1-alpha)
  }
}

VaRCACont_acc_div
VaRCAfr1_acc_div
VaRCAfr2_acc_div
CVaRCACont_acc_div
CVaRCAfr1_acc_div
CVaRCAfr2_acc_div

#VaR Total

VaRCATotal_acc_div=wquantile(PGT_acc_div,p_esc, 1-alpha)
CVaRCATotal_acc_div= wcvar(PGT_acc_div,p_esc,1-alpha)
VaRCATotalfr1_acc_div=wquantile(PGfr1T_acc_div,p_esc,1-alpha)
CVaRCATotalfr1_acc_div= wcvar(PGfr1T_acc_div,p_esc,1-alpha)
VaRCATotalfr2_acc_div=wquantile(PGfr2T_acc_div,p_esc,1-alpha)
CVaRCATotalfr2_acc_div= wcvar(PGfr2T_acc_div,p_esc,1-alpha)

print("Valor de acciones y divisas")
print(V0_acc_div )
print("Valor en Riesgo de acciones y divisas")
print(VaRCACont_acc_div)
print("Valor en Riesgo de divisas")
print(VaRCATotalfr1_acc_div)
print("Valor en Riesgo de acciones")
print(VaRCATotalfr2_acc_div)

print("Valor en Riesgo Condicional de acciones y divisas")
print(CVaRCACont_acc_div)
print("Valor en Riesgo Condicional de divisas")
print(CVaRCATotalfr1_acc_div)
print("Valor en Riesgo Condicional de acciones")
print(CVaRCATotalfr2_acc_div)

[1] "Valor de acciones y divisas"
   EURUSD.X  GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1:    16689 -16857.19 30830.55  -91250       73820     86592
[1] "Valor en Riesgo de acciones y divisas"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -112.8287 -148.7894 -335.7562 -1696.911 -2873.721 -2890.737
[1] "Valor en Riesgo de divisas"
[1] -371.7948
[1] "Valor en Riesgo de acciones"
[1] -4807.637
[1] "Valor en Riesgo Condicional de acciones y divisas"
        [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -173.32 -175.2626 -512.0436 -2835.522 -3147.862 -4130.248
[1] "Valor en Riesgo Condicional de divisas"
[1] -430.0354
[1] "Valor en Riesgo Condicional de acciones"
[1] -5847.692


#### Inciso 3: Cetes

In [48]:
%%R
m = 1
VaRCACont_cetes=matrix(0,1,m)
CVaRCACont_cetes=matrix(0,1,m)

for (i in (1:m))
{
  VaRCACont_cetes[i]=wquantile(as.matrix(PG_cetes[,i]), p_esc, 1-alpha)
  CVaRCACont_cetes[i]= wcvar(as.matrix(PG_cetes[,i]), p_esc, 1-alpha)
}


#VaR Total

print("Valor de cete")
print(V0_cetes[1,1])
print("Valor en Riesgo de cetes")
print(VaRCACont_cetes)
print("Valor en Riesgo Condicional de cetes")
print(CVaRCACont_cetes)


[1] "Valor de cete"
[1] 9.760299
[1] "Valor en Riesgo de cetes"
          [,1]
[1,] -8.784538
[1] "Valor en Riesgo Condicional de cetes"
          [,1]
[1,] -8.784705


#### Inciso 3: Bondes D

In [49]:
%%R
m=count(N_bd)
VaRCACont_bd=matrix(0,1,m)
VaRCAfr1_bd=matrix(0,1,m)
VaRCAfr2_bd=matrix(0,1,m)
VaRCAfr3_bd=matrix(0,1,m)
CVaRCACont_bd=matrix(0,1,m)
CVaRCAfr1_bd=matrix(0,1,m)
CVaRCAfr2_bd=matrix(0,1,m)
CVaRCAfr3_bd=matrix(0,1,m)

for (i in (1:m))
{
  VaRCACont_bd[i]=wquantile(as.matrix(PG_bd[,i]), p_esc, 1-alpha)
  CVaRCACont_bd[i]= wcvar(as.matrix(PG_bd[,i]), p_esc, 1-alpha)
  if (i<=m)
  {
  VaRCAfr1_bd[i]=wquantile(as.matrix(PGfr1_bd[,i]), p_esc, 1-alpha)
  CVaRCAfr1_bd[i]= wcvar(as.matrix(PGfr1_bd[,i]), p_esc, 1-alpha)
  VaRCAfr2_bd[i]=wquantile(as.matrix(PGfr2_bd[,i]), p_esc, 1-alpha)
  CVaRCAfr2_bd[i]= wcvar(as.matrix(PGfr2_bd[,i]), p_esc, 1-alpha)
  VaRCAfr3_bd[i]=wquantile(as.matrix(PGfr3_bd[,i]), p_esc, 1-alpha)
  CVaRCAfr3_bd[i]= wcvar(as.matrix(PGfr3_bd[,i]), p_esc, 1-alpha)
  }
}


#VaR Total

VaRCATotal_bd=wquantile(PGT_bd,p_esc, 1-alpha)
CVaRCATotal_bd= wcvar(PGT_bd,p_esc,1-alpha)
VaRCATotalfr1_bd=wquantile(PGfr1T_bd,p_esc,1-alpha)
CVaRCATotalfr1_bd= wcvar(PGfr1T_bd,p_esc,1-alpha)
VaRCATotalfr2_bd=wquantile(PGfr2T_bd,p_esc,1-alpha)
CVaRCATotalfr2_bd= wcvar(PGfr2T_bd,p_esc,1-alpha)
VaRCATotalfr3_bd=wquantile(PGfr3T_bd,p_esc,1-alpha)
CVaRCATotalfr3_bd= wcvar(PGfr3T_bd,p_esc,1-alpha)

print("Valor del bonde D")
print(V0_bd[1,1])
print("Valor en riesgo del bonde D")
print(VaRCACont_bd[1,1])
print("Valor en Riesgo Condicional de bondes D")
print(CVaRCACont_bd[1,1])
print(cbind(VaRCATotalfr1_bd,VaRCATotalfr2_bd,VaRCATotalfr3_bd))
print(cbind(CVaRCATotalfr1_bd,CVaRCATotalfr2_bd,CVaRCATotalfr3_bd))

[1] "Valor del bonde D"
[1] -97309.88
[1] "Valor en riesgo del bonde D"
[1] -1025.31
[1] "Valor en Riesgo Condicional de bondes D"
[1] NaN
     VaRCATotalfr1_bd VaRCATotalfr2_bd VaRCATotalfr3_bd
[1,]        -734.4095        -244.6142        -100.0845
     CVaRCATotalfr1_bd CVaRCATotalfr2_bd CVaRCATotalfr3_bd
[1,]               NaN         -334.3568         -132.3988


In [50]:
%%R

#VaR y CVaR total de bonos
VaRTCA_bonos = wquantile(PG_bonos,p_esc, 1-alpha)
CVaRTCA_bonos = wcvar(PG_bonos,p_esc,1-alpha)

print(cbind(VaRTCA_bonos, CVaRTCA_bonos))

     VaRTCA_bonos CVaRTCA_bonos
[1,]    -1034.094           NaN


#### Inciso 4: Forward de tipo de cambio

In [51]:
%%R
m=ncol(plazos_fwd)
#VaRCA por posición

VaRCACont_fwtdc=matrix(0,1,m)
VaRCAfr1_fwtdc=matrix(0,1,m)
VaRCAfr2_fwtdc=matrix(0,1,m)
VaRCAfr3_fwtdc=matrix(0,1,m)
CVaRCACont_fwtdc=matrix(0,1,m)
CVaRCAfr1_fwtdc=matrix(0,1,m)
CVaRCAfr2_fwtdc=matrix(0,1,m)
CVaRCAfr3_fwtdc=matrix(0,1,m)
for (i in (1:m))
{
  VaRCACont_fwtdc[i]=wquantile(PG_fwtdc[,i],p_esc,1-alpha)
  VaRCAfr1_fwtdc[i]=wquantile(PGfr1_fwtdc[,i],p_esc,1-alpha)
  VaRCAfr2_fwtdc[i]=wquantile(PGfr2_fwtdc[,i],p_esc,1-alpha)
  VaRCAfr3_fwtdc[i]=wquantile(PGfr3_fwtdc[,i],p_esc,1-alpha)
  CVaRCAfr1_fwtdc[i]= wcvar(PGfr1_fwtdc[,i],p_esc,1-alpha)
  CVaRCAfr2_fwtdc[i]= wcvar(PGfr2_fwtdc[,i],p_esc,1-alpha)
  CVaRCAfr3_fwtdc[i]= wcvar(PGfr3_fwtdc[,i],p_esc,1-alpha)
  CVaRCACont_fwtdc[i]= wcvar(PG_fwtdc[,i],p_esc,1-alpha)
}

VaRCACont_fwtdc
VaRCAfr1_fwtdc
VaRCAfr2_fwtdc
VaRCAfr3_fwtdc
CVaRCACont_fwtdc
CVaRCAfr1_fwtdc
CVaRCAfr2_fwtdc
CVaRCAfr3_fwtdc


#VaRCA Total
VaRCATotal_fwtdc=wquantile(PGT_fwtdc,p_esc,1-alpha)
CVaRCATotal_fwtdc= wcvar(PGT_fwtdc,p_esc,1-alpha)
VaRCATotalfr1_fwtdc=wquantile(PGfr1T_fwtdc,p_esc,1-alpha)
CVaRCATotalfr1_fwtdc= wcvar(PGfr1T_fwtdc,p_esc,1-alpha)
VaRCATotalfr2_fwtdc=wquantile(PGfr2T_fwtdc,p_esc,1-alpha)
CVaRCATotalfr2_fwtdc= wcvar(PGfr2T_fwtdc,p_esc,1-alpha)
VaRCATotalfr3_fwtdc=wquantile(PGfr3T_fwtdc,p_esc,1-alpha)
CVaRCATotalfr3_fwtdc= wcvar(PGfr3T_fwtdc,p_esc,1-alpha)


cbind(VaRCATotal_fwtdc,sum(V0_fwtdc), VaRCACont_fwtdc, V0_fwtdc)
print(V0_fwtdc)
print(VaRCACont_fwtdc)
cbind(CVaRCATotal_fwtdc,sum(V0_fwtdc), CVaRCACont_fwtdc, V0_fwtdc)
print(CVaRCACont_fwtdc)
print(CVaRCAfr1_fwtdc)
print(CVaRCAfr2_fwtdc)
print(CVaRCAfr3_fwtdc)
print(cbind(VaRCATotal_fwtdc,VaRCATotalfr1_fwtdc,VaRCATotalfr2_fwtdc,VaRCATotalfr3_fwtdc))
print(cbind(CVaRCATotal_fwtdc,CVaRCATotalfr1_fwtdc,CVaRCATotalfr2_fwtdc,CVaRCATotalfr3_fwtdc))

print("VaRCA por contrato")
print(VaRCACont_fwtdc)
#print("VaR por factor de riesgo 1 por contrato")
#print(VaRfr1_fwtdc)
#print("VaR por factor de riesgo 2 por contrato")
#print(VaRfr2_fwtdc)
#print("VaR por factor de riesgo 3 por contrato")
#print(VaRfr3_fwtdc)#-
print("VaRCA por factor de riesgo 1 total")
print(VaRCATotalfr1_fwtdc)
print("VaRCA por factor de riesgo 2 total")
print(VaRCATotalfr2_fwtdc)
print("VaRCA por factor de riesgo 3 total")
print(VaRCATotalfr3_fwtdc)
print("VaRCA Total")
print(VaRCATotal_fwtdc)

print("CVaRCA por contrato")
print(CVaRCACont_fwtdc)
#print("CVaR por factor de riesgo 1 por contrato")
#print(CVaRfr1_fwtdc)
#print("CVaR por factor de riesgo 2 por contrato")
#print(CVaRfr2_fwtdc)
#print("CVaR por factor de riesgo 3 por contrato")
#print(CVaRfr3_fwtdc)
print("CVaRCA por factor de riesgo 1 total")
print(CVaRCATotalfr1_fwtdc)
print("CVaRCA por factor de riesgo 2 total")
print(CVaRCATotalfr2_fwtdc)
print("CVaRCA por factor de riesgo 3 total")
print(CVaRCATotalfr3_fwtdc)
print("CVaRCA Total")
print(CVaRCATotal_fwtdc)


          [,1] [,2]
[1,] -26.92905    0
          [,1] [,2]
[1,] -22.37881    0
          [,1] [,2]
[1,] -34.12279  NaN
            [,1] [,2]
[1,] -0.00683885  NaN
             [,1] [,2]
[1,] -0.007188417  NaN
     [,1] [,2]
[1,]  NaN  NaN
     VaRCATotal_fwtdc VaRCATotalfr1_fwtdc VaRCATotalfr2_fwtdc
[1,]        -22.37881        -0.003381063        -0.006247545
     VaRCATotalfr3_fwtdc
[1,]            26.92905
     CVaRCATotal_fwtdc CVaRCATotalfr1_fwtdc CVaRCATotalfr2_fwtdc
[1,]         -34.12279          -0.00683885         -0.007188417
     CVaRCATotalfr3_fwtdc
[1,]                  NaN
[1] "VaRCA por contrato"
          [,1] [,2]
[1,] -22.37881    0
[1] "VaRCA por factor de riesgo 1 total"
[1] -0.003381063
[1] "VaRCA por factor de riesgo 2 total"
[1] -0.006247545
[1] "VaRCA por factor de riesgo 3 total"
[1] 26.92905
[1] "VaRCA Total"
[1] -22.37881
[1] "CVaRCA por contrato"
          [,1] [,2]
[1,] -34.12279  NaN
[1] "CVaRCA por factor de riesgo 1 total"
[1] -0.00683885
[1] "CVaRCA p

#### Inciso 4: Forwards de indice

In [52]:
%%R
m=ncol(plazos_fwd_ind)  
#VaRCA por posición
VaRCACont_fwind=matrix(0,1,m)
VaRCAfr1_fwind=matrix(0,1,m)
VaRCAfr2_fwind=matrix(0,1,m)
VaRCAfr3_fwind=matrix(0,1,m)
CVaRCACont_fwind=matrix(0,1,m)
CVaRCAfr1_fwind=matrix(0,1,m)
CVaRCAfr2_fwind=matrix(0,1,m)
CVaRCAfr3_fwind=matrix(0,1,m)
for (i in (1:m))
{
  VaRCACont_fwind[i]=wquantile(PG_fwind[,i],p_esc,1-alpha)
  VaRCAfr1_fwind[i]=wquantile(PGfr1_fwind[,i],p_esc,1-alpha)
  VaRCAfr2_fwind[i]=wquantile(PGfr2_fwind[,i],p_esc,1-alpha)
  VaRCAfr3_fwind[i]=wquantile(PGfr3_fwind[,i],p_esc,1-alpha)
  CVaRCAfr1_fwind[i]= wcvar(PGfr1_fwind[,i],p_esc,1-alpha)
  CVaRCAfr2_fwind[i]= wcvar(PGfr2_fwind[,i],p_esc,1-alpha)
  CVaRCAfr3_fwind[i]= wcvar(PGfr3_fwind[,i],p_esc,1-alpha)
  CVaRCACont_fwind[i]= wcvar(PG_fwind[,i],p_esc,1-alpha)
}
VaRCACont_fwind
VaRCAfr1_fwind
VaRCAfr2_fwind
CVaRCACont_fwind
CVaRCAfr1_fwind
CVaRCAfr2_fwind

#VaRCA Total

VaRCATotal_fwind=wquantile(PGT_fwind,p_esc,1-alpha)
CVaRCATotal_fwind= wcvar(PGT_fwind,p_esc,1-alpha)
VaRCATotalfr1_fwind=wquantile(PGfr1T_fwind,p_esc,1-alpha)
CVaRCATotalfr1_fwind= wcvar(PGfr1T_fwind,p_esc,1-alpha)
VaRCATotalfr2_fwind=wquantile(PGfr2T_fwind,p_esc,1-alpha)
CVaRCATotalfr2_fwind= wcvar(PGfr2T_fwind,p_esc,1-alpha)
VaRCATotalfr3_fwind=wquantile(PGfr3T_fwind,p_esc,1-alpha)
CVaRCATotalfr3_fwind= wcvar(PGfr3T_fwind,p_esc,1-alpha)

print(cbind(VaRCATotal_fwind,sum(V0_fwind), VaRCACont_fwind, V0_fwind))
print(cbind(CVaRCATotal_fwind,sum(V0_fwind), CVaRCACont_fwind, V0_fwind))
print(cbind(VaRCATotal_fwind,VaRCATotalfr1_fwind,VaRCATotalfr2_fwind,VaRCATotalfr3_fwind))
cbind(CVaRCATotal_fwind,CVaRCATotalfr1_fwind,CVaRCATotalfr2_fwind,CVaRCATotalfr3_fwind)

print("VaRCA por contrato")
print(VaRCACont_fwind)
#print("VaR por factor de riesgo 1 por contrato")
#print(VaRfr1_fwind)
#print("VaR por factor de riesgo 2 por contrato")
#print(VaRfr2_fwind)
#print("VaR por factor de riesgo 3 por contrato")
#print(VaRfr3_fwind)
print("VaRCA por factor de riesgo 1 total")
print(VaRCATotalfr1_fwind)
print("VaRCA por factor de riesgo 2 total")
print(VaRCATotalfr2_fwind)
print("VaRCA por factor de riesgo 3 total")
print(VaRCATotalfr3_fwind)
print("VaRCA Total")
print(VaRCATotal_fwind)

print("CVaRCA por contrato")
print(CVaRCACont_fwind)
#print("CVaR por factor de riesgo 1 por contrato")
#print(CVaRfr1_fwind)
#print("CVaR por factor de riesgo 2 por contrato")
#print(CVaRfr2_fwind)
#print("CVaR por factor de riesgo 3 por contrato")
#print(CVaRfr3_fwind)
print("CVaRCA por factor de riesgo 1 total")
print(CVaRCATotalfr1_fwind)
print("CVaRCA por factor de riesgo 2 total")
print(CVaRTotalfr2_fwind)
print("CVaRCA por factor de riesgo 3 total")
print(CVaRCATotalfr3_fwind)
print("CVaRCA Total")
print(CVaRCATotal_fwind)




     VaRCATotal_fwind                                
[1,]        -44186.72 148027.6 -44186.72 0 148027.6 0
     CVaRCATotal_fwind                                  
[1,]         -50371.71 148027.6 -50371.71 NaN 148027.6 0
     VaRCATotal_fwind VaRCATotalfr1_fwind VaRCATotalfr2_fwind
[1,]        -44186.72           -44186.72           -44186.72
     VaRCATotalfr3_fwind
[1,]           -44186.72
[1] "VaRCA por contrato"
          [,1] [,2]
[1,] -44186.72    0
[1] "VaRCA por factor de riesgo 1 total"
[1] -44186.72
[1] "VaRCA por factor de riesgo 2 total"
[1] -44186.72
[1] "VaRCA por factor de riesgo 3 total"
[1] -44186.72
[1] "VaRCA Total"
[1] -44186.72
[1] "CVaRCA por contrato"
          [,1] [,2]
[1,] -50371.71  NaN
[1] "CVaRCA por factor de riesgo 1 total"
[1] -50371.71
[1] "CVaRCA por factor de riesgo 2 total"
[1] -57376.18
[1] "CVaRCA por factor de riesgo 3 total"
[1] -50371.71
[1] "CVaRCA Total"
[1] -50371.71


In [53]:
%%R
VaRCAT_forwards = wquantile(PGT_forwards,p_esc,1-alpha)
CVaRCAT_forwards = wcvar(PGT_forwards,p_esc,1-alpha)
print("VaRCA de futuros")
print(VaRCAT_forwards)
print("CVaRCA de futuros")
print(CVaRCAT_forwards)

[1] "VaRCA de futuros"
[1] -44182.25
[1] "CVaRCA de futuros"
[1] -50374.8


#### Inciso 5: Swaps

In [54]:
%%R
m=ncol(N)
VaRCACont_sw=matrix(0,1,m)
VaRCAfr1_sw=matrix(0,1,m)
VaRCAfr2_sw=matrix(0,1,m)
CVaRCACont_sw=matrix(0,1,m)
CVaRCAfr1_sw=matrix(0,1,m)
CVaRCAfr2_sw=matrix(0,1,m)
for (i in (1:m))
{
  VaRCACont_sw[i]=wquantile(PG_sw[,i],p_esc,1-alpha)
  VaRCAfr1_sw[i]=wquantile(PGfr1_sw[,i],p_esc, 1-alpha)
  VaRCAfr2_sw[i]=wquantile(PGfr2_sw[,i],p_esc,1-alpha)
  CVaRCAfr1_sw[i]= wcvar(PGfr1_sw[,i],p_esc, 1-alpha)
  CVaRCAfr2_sw[i]= wcvar(PGfr2_sw[,i],p_esc,1-alpha)
  CVaRCACont_sw[i]= wcvar(PG_sw[,i],p_esc,1-alpha)
}

#VaRCA Total

VaRCATotal_sw=wquantile(PGT_sw,p_esc,1-alpha)
CVaRCATotal_sw= wcvar(PGT_sw,p_esc,1-alpha)
VaRCATotalfr1_sw=wquantile(PGfr1T_sw,p_esc,1-alpha)
CVaRCATotalfr1_sw= wcvar(PGfr1T_sw,p_esc,1-alpha)
VaRCATotalfr2_sw= wquantile(PGfr2T_sw,p_esc,1-alpha)
CVaRCATotalfr2_sw= wcvar(PGfr2T_sw,p_esc,1-alpha)

print("VaRCA por contrato")
print(VaRCACont_sw)
print("VaRCA por factor de riesgo 1 por contrato")
print(VaRCAfr1_sw)
print("VaRCA por factor de riesgo 2 por contrato")
print(VaRCAfr2_sw)
print("VaRCA por factor de riesgo 1 total")
print(VaRCATotalfr1_sw)
print("VaRCA por factor de riesgo 2 total")
print(VaRCATotalfr2_sw)
print("VaRCA Total")
print(VaRCATotal_sw)

print("CVaRCA por contrato")
print(CVaRCACont_sw)
print("CVaRCA por factor de riesgo 1 por contrato")
print(CVaRCAfr1_sw)
print("CVaRCA por factor de riesgo 2 por contrato")
print(CVaRCAfr2_sw)
print("CVaRCA por factor de riesgo 1 total")
print(CVaRCATotalfr1_sw)
print("CVaRCA por factor de riesgo 2 total")
print(CVaRCATotalfr2_sw)
print("CVaRCA Total")
print(CVaRCATotal_sw)


[1] "VaRCA por contrato"
           [,1]       [,2]
[1,] -0.9787824 -0.1520867
[1] "VaRCA por factor de riesgo 1 por contrato"
           [,1]       [,2]
[1,] -0.9788261 -0.1520722
[1] "VaRCA por factor de riesgo 2 por contrato"
              [,1]          [,2]
[1,] -0.0005592399 -4.159833e-05
[1] "VaRCA por factor de riesgo 1 total"
[1] -1.130898
[1] "VaRCA por factor de riesgo 2 total"
[1] -0.0005985692
[1] "VaRCA Total"
[1] -1.130869
[1] "CVaRCA por contrato"
          [,1]       [,2]
[1,] -1.195972 -0.2270094
[1] "CVaRCA por factor de riesgo 1 por contrato"
          [,1]       [,2]
[1,] -1.196102 -0.2269802
[1] "CVaRCA por factor de riesgo 2 por contrato"
              [,1]          [,2]
[1,] -0.0007944159 -6.761437e-05
[1] "CVaRCA por factor de riesgo 1 total"
[1] -1.253598
[1] "CVaRCA por factor de riesgo 2 total"
[1] -0.0008639393
[1] "CVaRCA Total"
[1] -1.253477


#### Inciso 6: Opciones

In [55]:
%%R
m=ncol(plazos_oir)
VaRCACont_oir=matrix(0,1,m)
VaRCAfr1_oir=matrix(0,1,m)
VaRCAfr2_oir=matrix(0,1,m)
VaRCAfr3_oir=matrix(0,1,m)
CVaRCACont_oir=matrix(0,1,m)
CVaRCAfr1_oir=matrix(0,1,m)
CVaRCAfr2_oir=matrix(0,1,m)
CVaRCAfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCACont_oir[i]=wquantile(PG_oir[,i],p_esc,1-alpha)
  VaRCAfr1_oir[i]=wquantile(PGfr1_oir[,i],p_esc,1-alpha)
  VaRCAfr2_oir[i]=wquantile(PGfr2_oir[,i],p_esc,1-alpha)
  VaRCAfr3_oir[i]=wquantile(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCAfr1_oir[i]= wcvar(PGfr1_oir[,i],p_esc,1-alpha)
  CVaRCAfr2_oir[i]= wcvar(PGfr2_oir[,i],p_esc,1-alpha)
  CVaRCAfr3_oir[i]= wcvar(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCACont_oir[i]= wcvar(PG_oir[,i],p_esc,1-alpha)
}


#VaRCA Total
VaRCATotal_oir=wquantile(PGT_oir,p_esc,1-alpha)
CVaRCATotal_oir= wcvar(PGT_oir,p_esc,1-alpha)
VaRCATotalfr1_oir=wquantile(PGfr1T_oir,p_esc,1-alpha)
CVaRCATotalfr1_oir= wcvar(PGfr1T_oir,p_esc,1-alpha)
VaRCATotalfr2_oir=wquantile(PGfr2T_oir,p_esc,1-alpha)
CVaRCATotalfr2_oir= wcvar(PGfr2T_oir,p_esc,1-alpha)
VaRCATotalfr3_oir=wquantile(PGfr3T_oir,p_esc,1-alpha)
CVaRCATotalfr3_oir= wcvar(PGfr3T_oir,p_esc,1-alpha)


#print(cbind(VaRCATotal_oir,sum(V0_oir), VaRCACont_oir, V0_oir))
#print(cbind(CVaRCATotal_oir,sum(V0_oir), CVaRCACont_oir, V0_oir))
#print(cbind(VaRCATotal_oir,VaRCATotalfr1_oir,VaRCATotalfr2_oir,VaRCATotalfr3_oir))
#print(cbind(CVaRCATotal_oir,CVaRCATotalfr1_oir,CVaRCATotalfr2_oir,CVaRCATotalfr3_oir))

print(V0_oir)
print("VaRCA por contrato")
print(VaRCACont_oir)
print("VaRCA por factor de riesgo 1 por contrato")
print(VaRCAfr1_oir)
print("VaRCA por factor de riesgo 2 por contrato")
print(VaRCAfr2_oir)
print("VaRCA por factor de riesgo 3 por contrato")
print(VaRCAfr3_oir)
print("VaRCA por factor de riesgo 1 total")
print(VaRCATotalfr1_oir)
print("VaRCA por factor de riesgo 2 total")
print(VaRCATotalfr2_oir)
print("VaRCA por factor de riesgo 3 total")
print(VaRCATotalfr3_oir)
print("VaRCA Total")
print(VaRCATotal_oir)

print("CVaRCA por contrato")
print(CVaRCACont_oir)
print("CVaRCA por factor de riesgo 1 por contrato")
print(CVaRCAfr1_oir)
print("CVaRCA por factor de riesgo 2 por contrato")
print(CVaRCAfr2_oir)
print("CVaRCA por factor de riesgo 3 por contrato")
print(CVaRCAfr3_oir)
print("CVaRCA por factor de riesgo 1 total")
print(CVaRCATotalfr1_oir)
print("CVaRCA por factor de riesgo 2 total")
print(CVaRCATotalfr2_oir)
print("CVaRCA por factor de riesgo 3 total")
print(CVaRCATotalfr3_oir)
print("CVaRCA Total")
print(CVaRCATotal_oir)

         [,1]       [,2]
[1,] 1.422881 0.05877815
[1] "VaRCA por contrato"
           [,1]         [,2]
[1,] -0.0603302 -0.007003424
[1] "VaRCA por factor de riesgo 1 por contrato"
             [,1]          [,2]
[1,] -0.004132626 -4.751446e-05
[1] "VaRCA por factor de riesgo 2 por contrato"
            [,1]         [,2]
[1,] -0.05650327 -0.006978037
[1] "VaRCA por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]    0    0
[1] "VaRCA por factor de riesgo 1 total"
[1] -0.004156333
[1] "VaRCA por factor de riesgo 2 total"
[1] -0.05193862
[1] "VaRCA por factor de riesgo 3 total"
[1] 0
[1] "VaRCA Total"
[1] -0.05578978
[1] "CVaRCA por contrato"
            [,1]        [,2]
[1,] -0.09920038 -0.01110537
[1] "CVaRCA por factor de riesgo 1 por contrato"
             [,1]         [,2]
[1,] -0.005414904 -5.86281e-05
[1] "CVaRCA por factor de riesgo 2 por contrato"
            [,1]        [,2]
[1,] -0.09716872 -0.01109832
[1] "CVaRCA por factor de riesgo 3 por contrato"
     [,1] [,2]
[1,]  N

## Riesgo integral, sin alisado

In [56]:
%%R
#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
PGPort_ACC=PGfr2T_acc_div + PGfr3T_fwind #Pérdidas y ganancias
VaRPort_ACC=equantile(PGPort_ACC,1-alpha,Ns) #VaR
CVaRPort_ACC= mean(PGPort_ACC[which(PGPort_ACC<VaRPort_ACC)]) #CVaR


#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc
PGPort_TI=PGT_bd+PGT_sw+PGfr1T_oir +PGfr2T_oir + PGfr1T_fwind +PGfr2T_fwind + PGfr1T_fwtdc + PGfr2T_fwtdc + PG_cetes#Pérdidas y ganancias
VaRPort_TI=equantile(PGPort_TI,1-alpha,Ns) #VaR
CVaRPort_TI= mean(PGPort_TI[which(PGPort_TI<VaRPort_TI)]) #CVaR

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc
PGPort_TDC=PGfr1T_acc_div + PGfr3T_fwtdc  #Pérdidas y ganancias
VaRPort_TDC=equantile(PGPort_TDC,1-alpha,Ns) #VaR
CVaRPort_TDC= mean(PGPort_TDC[which(PGPort_TDC<VaRPort_TDC)]) #CVaR

#Volatilidad
#1. Sólo aplica la volatilidad de Opciones de tasa de interés

PGPort_VOL=PGfr3T_oir  #Pérdidas y ganancias
VaRPort_VOL=equantile(PGPort_VOL,1-alpha,Ns) #VaR
CVaRPort_VOL= mean(PGPort_VOL[which(PGPort_VOL<VaRPort_VOL)]) #CVaR


#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos

PGT_Port=PGPort_ACC+PGPort_TI+PGPort_TDC+PGPort_VOL
VaRTotal_Port=equantile(PGT_Port,1-alpha,Ns) #VaR
CVaRTotal_Port= mean(PGT_Port[which(PGT_Port<VaRTotal_Port)]) #CVaR

print('VaR, CVaR acciones')
print(cbind(VaRPort_ACC, CVaRPort_ACC))
print('VaR, CVaR tasa interes')
print(cbind(VaRPort_TI, CVaRPort_TI))
print('VaR, CVaR tipo de cambio')
print(cbind(VaRPort_TDC, CVaRPort_TDC))
print('VaR, CVaR volatilidad')
print(cbind(VaRPort_VOL, CVaRPort_VOL))
print('VaR, CVaR portafolio')
print(cbind(VaRTotal_Port, CVaRTotal_Port))

[1] "VaR, CVaR acciones"
     VaRPort_ACC CVaRPort_ACC
[1,]   -48913.22    -60414.48
[1] "VaR, CVaR tasa interes"
     VaRPort_TI CVaRPort_TI
[1,]   -95166.7   -114725.6
[1] "VaR, CVaR tipo de cambio"
     VaRPort_TDC CVaRPort_TDC
[1,]    -532.947    -551.4357
[1] "VaR, CVaR volatilidad"
     VaRPort_VOL CVaRPort_VOL
[1,]           0          NaN
[1] "VaR, CVaR portafolio"
     VaRTotal_Port CVaRTotal_Port
[1,]       -143498      -175243.1


## Riesgo integral, con alisado

In [57]:
%%R
#Medición de riesgo por factor de riesgo de todo el portafolios
#Acciones
#1. Acciones 
VaRCAPort_ACC=wquantile(PGPort_ACC,p_esc,1-alpha) #VaRCA
CVaRCAPort_ACC= wcvar(PGPort_ACC,p_esc,1-alpha) #CVaRCA


#Tasa de Interés
#1. Dado que swaps y bondes son de tasa de interés usaremos PGT_bd y PGT_sw
#2. Para futuros usaremos PGfr1T_fwtdc y PGfr2T_fwtdc

VaRCAPort_TI=wquantile(PGPort_TI,p_esc,1-alpha) #VaRCA
CVaRCAPort_TI= wcvar(PGPort_TI,p_esc,1-alpha) #CVaRCA

#Tipo de cambio
#1. Dado que swaps y bondes son de tasa de interés no usamos nada
#2. Para futuros usamos sólo PGfr3T_fwtdc

VaRCAPort_TDC=wquantile(PGPort_TDC,p_esc,1-alpha) #VaRCA
CVaRCAPort_TDC= wcvar(PGPort_TDC,p_esc,1-alpha) #CVaRCA

#Volatilidad
#1. Sólo aplica la volatilidad de Opciones de tasa de interés

VaRCAPort_VOL=wquantile(PGPort_VOL,p_esc,1-alpha) #VaRCA
CVaRCAPort_VOL= wcvar(PGPort_VOL,p_esc,1-alpha) #CVaRCA


#Medición de riesgo de todo el portafolios
#Sumar todos los PGT de todos los instrumentos


VaRCATotal_Port=wquantile(PGT_Port,p_esc,1-alpha) #VaRCA
CVaRCATotal_Port= wcvar(PGT_Port,p_esc,1-alpha) #CVaRCA

print('VaRCA, CVaRCA acciones')
print(cbind(VaRCAPort_ACC, CVaRCAPort_ACC))
print('VaRCA, CVaRCA tasa interes')
print(cbind(VaRCAPort_TI, CVaRCAPort_TI))
print('VaRCA, CVaRCA tipo de cambio')
print(cbind(VaRCAPort_TDC, CVaRCAPort_TDC))
print('VaRCA, CVaRCA volatilidad')
print(cbind(VaRCAPort_VOL, CVaRCAPort_VOL))
print('VaRCA, CVaRCA portafolio')
print(cbind(VaRCATotal_Port, CVaRCATotal_Port))

[1] "VaRCA, CVaRCA acciones"
     VaRCAPort_ACC CVaRCAPort_ACC
[1,]     -40315.83      -51858.44
[1] "VaRCA, CVaRCA tasa interes"
     VaRCAPort_TI CVaRCAPort_TI
[1,]    -88091.72       -100705
[1] "VaRCA, CVaRCA tipo de cambio"
     VaRCAPort_TDC CVaRCAPort_TDC
[1,]     -344.8658      -403.1063
[1] "VaRCA, CVaRCA volatilidad"
     VaRCAPort_VOL CVaRCAPort_VOL
[1,]             0            NaN
[1] "VaRCA, CVaRCA portafolio"
     VaRCATotal_Port CVaRCATotal_Port
[1,]       -128350.9        -152568.8
